<a href="https://colab.research.google.com/github/lalanikarim/notebooks/blob/main/Spoil_a_wish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spoil-A-Wish with LangChain, Cloudflare Workers AI, and Gradio

Spoil-A-Wish is a fun little game you play with friends where you make a wish and your friend grants it with a twist, essentially ruining it.

For example:

```
Wish: I wish for a million dollars.  
Response: Granted, but you get a million dollars in ZWL (Zimbabwe's defunct currency) which equals $15 USD. Enjoy your happy meal.
```

In this sample project, we will create a simple AI Agent that spoils your wishes.

We will build it with [LangChain](https://www.langchain.com) using [llama 3.1 70b instruct](https://developers.cloudflare.com/workers-ai/models/llama-3.1-70b-instruct/) model hosted on [Cloudflare Workers AI](https://developers.cloudflare.com/workers-ai/).  

`llama 3.1 70b instruct` model is available for free by Cloudflare while it is still in `beta` status. Other models are also available with [daily free token allocations](https://developers.cloudflare.com/workers-ai/platform/pricing/#free-allocation).

You will need your Cloudflare Account ID and API Token for this exercise.  
You can register for a free Cloudflare Account [https://www.cloudflare.com/](https://www.cloudflare.com/).

Account ID can be found at [https://dash.cloudflare.com](https://dash.cloudflare.com) under the `Overview` section towards the bottom. You may have to select a domain if you have multiple domains on your account.

API Token can be created [https://dash.cloudflare.com/profile/api-tokens](https://dash.cloudflare.com/profile/api-tokens). Make sure to give it permissions for `Workers AI`.

List of all availabe models supported by `Cloudflare Workers AI` can be found at [https://developers.cloudflare.com/workers-ai/models/](https://developers.cloudflare.com/workers-ai/models/).  
Select `Text Generation` under `Model Types`.

## 1. Install python dependencies

In [1]:
!pip install -q langchain-core langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.0 MB/s eta 0:00:00


## 2. Import packages

In [38]:
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.output_parsers import BaseOutputParser
from langchain_community.chat_models.cloudflare_workersai import ChatCloudflareWorkersAI
import json
import ast

## 3. Extract Google Colab Secrets

In [3]:
import os
from google.colab import userdata

CLOUDFLARE_ACCOUNT_ID = userdata.get('CLOUDFLARE_ACCOUNT_ID')
CLOUDFLARE_API_TOKEN = userdata.get('CLOUDFLARE_API_TOKEN')

if not CLOUDFLARE_ACCOUNT_ID or not CLOUDFLARE_API_TOKEN:
    raise ValueError("CLOUDFLARE_ACCOUNT_ID and CLOUDFLARE_API_TOKEN must be set in environment variables or userdata.")

os.environ['CLOUDFLARE_ACCOUNT_ID'] = CLOUDFLARE_ACCOUNT_ID
os.environ['CLOUDFLARE_API_TOKEN'] = CLOUDFLARE_API_TOKEN

## 4. Prepare Prompt Template

In [4]:
# prompt: create a prompt template for a game of "Spoil a wish". In this game the user asks for a wish and the ai grants the wish but spoils it.

prompt_template = """
You are playing the game "Spoil a Wish".  A user will make a wish. Your job is to grant their wish, but then spoil it in a creative and humorous way.  Make sure to follow these rules:

1. First, acknowledge and grant the wish in a positive and enthusiastic manner.
2. Then, introduce a twist or complication that completely ruins or undermines the wish in a funny way. Be creative with the way the wish is ruined.
3. Your response should be concise and engaging.

Here is the wish:

{wish}
"""

prompt = PromptTemplate.from_template(prompt_template)

## 5. Initialize LLM and create chain

In [5]:
# prompt: Initialize an LLM from langchain_community.chat_models.cloudflare_workersai.ChatCloudflareWorkersAI. Use CLOUDFLARE_ACCOUNT_ID and CLOUDFLARE_API_TOKEN from google.colab.userdata

model = ChatCloudflareWorkersAI(account_id=os.getenv('CLOUDFLARE_ACCOUNT_ID'), api_token=os.getenv('CLOUDFLARE_API_TOKEN'),model='@cf/meta/llama-3.1-70b-instruct')

# Example usage (you can replace "I wish for a million dollars" with any wish)
wish = "I wish for a million dollars"

chat = prompt | model

### Test chain

In [6]:
response: AIMessage = chat.invoke({"wish": wish})
response.content

'{\'result\': {\'response\': "You\'ve been granted ONE MILLION DOLLARS!!!\\n\\nHowever, due to a bizarre tax law from the 19th century that was never repealed, the government has decided to collect the entire amount in the form of pennies... and they need to be delivered by trained carrier pigeons. Your house will be filled with 50,000,000 pennies (yes, that\'s 250 tons of copper) and you\'ll have to care for 10,000 roller-eyed pigeons that refuse to fly unless they\'re listening to accordion music."}, \'success\': True, \'errors\': [], \'messages\': []}'

### Convert Cloudflare Workers AI response

In [7]:
ast.literal_eval(response.content)

{'result': {'response': "You've been granted ONE MILLION DOLLARS!!!\n\nHowever, due to a bizarre tax law from the 19th century that was never repealed, the government has decided to collect the entire amount in the form of pennies... and they need to be delivered by trained carrier pigeons. Your house will be filled with 50,000,000 pennies (yes, that's 250 tons of copper) and you'll have to care for 10,000 roller-eyed pigeons that refuse to fly unless they're listening to accordion music."},
 'success': True,
 'errors': [],
 'messages': []}

## 6. Create Custom Output Parser for Cloudflare Workers AI Response

In [8]:
from pydantic import BaseModel, Field
from typing import List

class CFWAIResult(BaseModel):
  response: str = Field(..., description="The response content.")

class CFWAIResponseMessage(BaseModel):
  success: bool = Field(description="Indicates if the operation was successful.")
  errors: List[str] = Field(default_factory=list, description="A list of errors encountered.")
  messages: List[str] = Field(default_factory=list, description="A list of informational messages.")
  result: CFWAIResult = Field(description="The result of the operation.")

class CFWAIResponseParser(BaseOutputParser):
    def parse(self, text: str) -> CFWAIResponseMessage:
        return CFWAIResponseMessage.parse_obj(ast.literal_eval(text))

### Create new chain with Custom Output Parser

In [9]:
chat = prompt | model | CFWAIResponseParser()

In [10]:
response: CFWAIResponseMessage = chat.invoke({"wish": wish})
print(response.result.response)

WOW, Congratulations! You now have ONE MILLION DOLLARS deposited into your bank account! I hope you're ready to live the high life!

However... there's a tiny catch. The million dollars is in Zimbabwean dollars, which, due to a crazy inflation rate, is roughly equivalent to about $5 USD. And to make matters worse, the only place you can use this fortune is at a small, obscure shop in rural Zimbabwe that only sells rubber chickens. Enjoy shopping!


In [11]:
response.result.response

"WOW, Congratulations! You now have ONE MILLION DOLLARS deposited into your bank account! I hope you're ready to live the high life!\n\nHowever... there's a tiny catch. The million dollars is in Zimbabwean dollars, which, due to a crazy inflation rate, is roughly equivalent to about $5 USD. And to make matters worse, the only place you can use this fortune is at a small, obscure shop in rural Zimbabwe that only sells rubber chickens. Enjoy shopping!"

## 7. Build a UI with Gradio

In [12]:
!pip install -q gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.2/168.2 kB 12.0 MB/s eta 0:00:00


In [37]:
import gradio as gr

def spoil_wish(wish):
    response: CFWAIResponseMessage = chat.invoke({"wish": wish})
    return response.result.response

with gr.Blocks(css="""
    .gradio-container {
        display: flex;
        flex-direction: column;
        max-width: 1000px !important;
    }
    .btn.success {
        background-color: #4CAF50;
        color: white;
    }
    .btn.danger {
        background-color: #F44336;
        color: white;
    }
""") as demo:
    gr.HTML("""
    <center>
      <h1>Spoil-A-Wish</h1>
    </center>
    """)
    with gr.Row():
        with gr.Column():
            wish = gr.TextArea(label="Wish")
            response = gr.TextArea(label="Response", visible=False)
            submit = gr.Button("Spoil Wish", elem_classes="btn success")
            reset = gr.Button("Try Again!", elem_classes="btn danger", visible=False)

    gr.Examples(
        examples=[
            ["I wish for a million dollars"],
            ["I wish for a new car"],
            ["I wish for a new house"],
            ["I wish for a new job"]
        ],
        inputs=[wish],
        label="Try these"

    )

    gr.on(
        triggers=submit.click,
        outputs=[wish, response, submit],
        fn=lambda: [gr.TextArea(interactive=False), gr.TextArea(visible=True), gr.Button(visible=False)],
        api_name=False
    ).then(
        fn=spoil_wish,
        inputs=wish,
        outputs=response
    ).then(
        outputs=reset,
        fn=lambda: gr.Button(visible=True),
        api_name=False
    )
    gr.on(
        triggers=reset.click,
        outputs=[response, reset, wish, submit],
        fn=lambda: [gr.TextArea(visible=False), gr.Button(visible=False), gr.TextArea(interactive=True, value=None), gr.Button(visible=True)],
        api_name=False
    )

demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://44ef81d9adaafc931e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
